In [1]:
import os
os.chdir('../..')

In [14]:
import numpy as np
import json
import pandas as pd
from datetime import datetime

from classes.helpers import *

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

data_folder = '../data/'

# Compute yearly z-score

In [6]:
df = pd.read_csv(data_folder + 'matched/beers.csv', header=[0,1])

beers_ids = {'ba': np.array(df['ba']['beer_id']), 'rb': np.array(df['rb']['beer_id'])}

In [21]:
ratings_year = {'ba': {}, 'rb': {}}

for key in ratings_year.keys():
    print('Parsing {} reviews.'.format(key.upper()))
    
    gen = parse(data_folder + key + '/ratings.txt.gz')
    
    for item in gen:
        
        date = int(item['date'])
        year = datetime.fromtimestamp(date).year
        
        beer_id = int(item['beer_id'])
        
        if beer_id in beers_ids[key]:
        
            if year not in ratings_year[key].keys():
                ratings_year[key][year] = []

            ratings_year[key][year].append(float(item['rating']))

Parsing BA reviews.
Parsing RB reviews.


In [24]:
z_score_params = {}
for key in ratings_year.keys():
    z_score_params[key] = {}
    for y in ratings_year[key].keys():
        z_score_params[key][y] = {'mean': np.mean(ratings_year[key][y]), 'std': np.std(ratings_year[key][y])}
        if z_score_params[key][y]['std'] == 0:
            z_score_params[key][y]['std'] = 1

In [29]:
z_score_params['ba'][1996] = {'mean': 0, 'std': 1}
z_score_params['ba'][1997] = {'mean': 0, 'std': 1}

In [31]:
with open(data_folder + 'tmp/z_score_params_matched.json', 'w') as file:
    json.dump(z_score_params, file)